<div style="text-align: center;"> <h1> FURIA Know Your Fan </h1> </div>

<div style="text-align: center;"> <h3> Todos os pip, ferramentas e bibliotecas utilizadas </h3> </div>

- pip install python-dotenv ipywidgets streamlit

<div style="text-align: center;"> <h3> Chave API Oculta (Arquivo .env) </h3> </div>

In [ ]:
from dotenv import load_dotenv
import os

# Carregar variáveis do .env
load_dotenv()

chave_api = os.getenv('API_KEY')

<div style="text-align: center;"> <h3>Coleta de Dados Básicos e Interesses Relacionados à FURIA</h3> </div>
  
- **Formulário Inicial:** Criar um formulário em Jupyter Notebook (usando `ipywidgets` ou `Streamlit`) para capturar dados pessoais fundamentais: nome, CPF, endereço, data de nascimento, email etc. Validar formato de CPF (biblioteca `python-bcpf` ou validação via regex) e outros campos para evitar erros de digitação.  

- **Interesses e Atividades em e-sports:** Incluir no notebook seções ou perguntas interativas sobre interesses em e-sports, times preferidos (FURIA), jogos mais acompanhados, frequência de eventos assistidos, ingressos ou periféricos adquiridos no último ano. Utilizar `pandas` para estruturar as respostas em um DataFrame. Poder-se-á simular coleta de dados de APIs públicas de eventos (por exemplo, dados de torneios CS:GO) ou pedir ao usuário que importe seu histórico (como um CSV de compras) para preencher esse perfil de maneira realista.  

- **Compras Relacionadas:** Se for aplicável, permitir o upload de extratos simplificados ou listas de compras (como merch de teams e-sports). Usar `pandas` ou `openpyxl` para ler esses arquivos e filtrar itens de interesse (palavras-chave relacionadas a jogos, marcas de e-sports, etc).

In [ ]:
                                                        # Formulário Via Notepad

# Importando bibliotecas necessárias
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import re

# Função para validar CPF
def validar_cpf(cpf):
    cpf = re.sub('[^0-9]', '', cpf)
    if len(cpf) != 11 or cpf == cpf[0] * 11:
        return False
    return True

# Widgets para coleta de dados básicos
nome = widgets.Text(description="Nome:")
cpf = widgets.Text(description="CPF:")
endereco = widgets.Text(description="Endereço:")
email = widgets.Text(description="Email:")

# Interesses focados na FURIA
jogos_furia = widgets.SelectMultiple(
    options=['CS:GO', 'Valorant', 'League of Legends', 'Rocket League', 'Outro'],
    description='Jogos FURIA'
)

produtos_furia = widgets.SelectMultiple(
    options=['Camisa', 'Boné', 'Mousepad', 'Moletons', 'Outro'],
    description='Produtos'
)

eventos_furia = widgets.RadioButtons(
    options=['Nunca participei', 'Eventos online', 'Eventos presenciais', 'Ambos'],
    description='Eventos'
)

# Upload de histórico de compras
upload_compras = widgets.FileUpload(
    accept='.csv, .xlsx', multiple=False, description='Upload Compras'
)

# Botão para salvar dados
botao_salvar = widgets.Button(description="Salvar Dados", button_style='success')

# Função para salvar e mostrar os dados
def salvar_dados(b):
    clear_output()
    if not validar_cpf(cpf.value):
        print("CPF inválido! Corrija e tente novamente.")
        display(cpf)
        return
    
    dados_usuario = {
        'Nome': nome.value,
        'CPF': cpf.value,
        'Endereço': endereco.value,
        'Email': email.value,
        'Jogos da FURIA que acompanha': jogos_furia.value,
        'Produtos da FURIA adquiridos': produtos_furia.value,
        'Participação em eventos da FURIA': eventos_furia.value
    }
    
    df_usuario = pd.DataFrame([dados_usuario])
    display(df_usuario)
    
    # Se houver upload de compras
    if upload_compras.value:
        print("\nArquivo de compras recebido! Analisando...\n")
        for filename, fileinfo in upload_compras.value.items():
            content = fileinfo['content']
            try:
                if filename.endswith('.csv'):
                    compras_df = pd.read_csv(pd.io.common.BytesIO(content))
                elif filename.endswith('.xlsx'):
                    compras_df = pd.read_excel(pd.io.common.BytesIO(content))
                else:
                    print("Formato de arquivo não suportado.")
                    return
                # Procurar compras relacionadas à FURIA
                palavras_chave = ['FURIA', 'furia.gg', 'camisa furia', 'boné furia', 'e-sports furia']
                compras_filtradas = compras_df[
                    compras_df.apply(lambda linha: any(palavra.lower() in str(linha).lower() for palavra in palavras_chave), axis=1)
                ]
                print("Compras relacionadas à FURIA encontradas:")
                display(compras_filtradas)
            except Exception as e:
                print(f"Erro ao processar arquivo: {e}")

botao_salvar.on_click(salvar_dados)

# Exibir widgets
display(nome, cpf, endereco, email, jogos_furia, produtos_furia, eventos_furia, upload_compras, botao_salvar)


In [ ]:
                                                # Rodando no Navegador

import http.server
import socketserver
import webbrowser
import json
import uuid
import os

PORT = 8080
DATA_DIR = "form_data"

class MyHandler(http.server.SimpleHTTPRequestHandler):
    def do_POST(self):
        if self.path == '/submit':
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length)

            # Decodificar os dados recebidos
            post_data = post_data.decode('utf-8')

            # Gerar ID único
            user_id = str(uuid.uuid4())

            # Salvar os dados crus em JSON
            if not os.path.exists(DATA_DIR):
                os.makedirs(DATA_DIR)
            
            file_path = os.path.join(DATA_DIR, f"{user_id}.json")
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump({'raw_data': post_data}, f, indent=4, ensure_ascii=False)  # Salvar como um dicionário com a chave 'raw_data'

            print(f"\n✅ Dados crus salvos em: {file_path}")

            # Enviar resposta simples de sucesso
            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps({'status': 'success', 'message': 'Dados recebidos e salvos com sucesso!'}).encode('utf-8'))
        else:
            self.send_error(404)

def run_server():
    with socketserver.TCPServer(("", PORT), MyHandler) as httpd:
        print(f"Servidor rodando em http://localhost:{PORT}")
        webbrowser.open(f"http://localhost:{PORT}/form.html")
        httpd.serve_forever()

if __name__ == "__main__":
    run_server()

<div style="text-align: center;"> <h3>Validação de Identidade com Abordagem de IA</h3> </div>

- **Upload de Documentos:** Incluir um widget de upload de arquivos de imagem (RG, CNH ou passaporte) e, opcionalmente, uma selfie do usuário.  
- **OCR e Extração de Dados:** Utilizar uma biblioteca de OCR como `pytesseract` ou `easyocr` para extrair texto do documento. Comparar nome e CPF extraídos com os dados básicos fornecidos para consistência.  
- **Reconhecimento Facial:** Para fortalecer a validação, aplicar uma técnica de reconhecimento facial simples. Usar bibliotecas como `face_recognition` ou `OpenCV` com modelos pré-treinados para detectar rostos na selfie e na foto do documento, e então verificar se pertencem à mesma pessoa (por exemplo, comparando descritores faciais).  
- **Feedback de Validação:** Exibir no notebook resultados da validação (válido/inválido) com base na correspondência de texto e face. Fornecer mensagens orientativas caso haja inconsistências (ex: “CPF não confere com o documento enviado”).

<div style="text-align: center;"> <h3>Integração com Redes Sociais (Simulada)</h3> </div>

- **Linkagem de Contas:** Na ausência de integrações reais, simular o processo solicitando ao usuário os nomes de usuário ou perfis em redes sociais (Twitter, Facebook, Instagram). Criar células de código comentadas que demonstram como usar APIs (ex: `tweepy` para Twitter, `facebook-sdk` para Facebook) para conectar as contas.  
- **Extração de Atividades Relacionadas a e-sports:** Para cada rede social simulada, mostrar como coletar dados relevantes: por exemplo, obter os últimos *tweets* do usuário e filtrar menções a e-sports ou FURIA; listar as páginas seguidas no Facebook com temas de gaming; ou verificar hashtags usadas em posts do Instagram. Usar `requests` e `BeautifulSoup` ou clientes de API para simulação. Armazenar essas informações em DataFrames para análise.  
- **Monitoramento de Interações:** Demonstrar código que analisa curtidas, comentários ou retweets em publicações de e-sports (fingindo as permissões de API). Por exemplo, usar `tweepy` para buscar tweets que mencionem FURIA ou eventos de e-sports que o usuário retuitou ou comentou.  

<div style="text-align: center;"> <h3>Enriquecimento de Perfil com Dados Sociais e Multimídia</h3> </div>

- **Análise de Comentários:** Para integrar comentários prévios do usuário no YouTube, Reddit e Twitter, incluir blocos que consumam APIs ou dados locais de análise anterior (supondo que existam). Usar `google-api-python-client` para extrair comentários de vídeos de e-sports do YouTube, `PRAW` para posts/comentários no Reddit, e `tweepy` ou dados simulados para tweets.  
- **Processamento de Linguagem Natural:** Aplicar NLP para entender o perfil do usuário: usar bibliotecas como `transformers` ou `spaCy` para classificar sentimento, identificar tópicos ou palavras-chave frequentes nesses comentários. Por exemplo, gerar um gráfico de palavras-chave mais mencionadas em e-sports, ou uma análise de sentimento geral sobre jogos específicos.  
- **Integração de Informações:** Combinar esses insights com os interesses declarados pelo usuário. Exibir visualmente (via `matplotlib` ou `seaborn`) uma nuvem de palavras ou gráfico que mostre as categorias de e-sports mais relevantes para o perfil (baseado em interesses + análise de comentários).  
- **Perfis em Sites de e-Sports:** Permitir que o usuário insira links para seus perfis em plataformas de e-sports (como GameBattles, HLTV, Liquipedia). Usar `requests` e `BeautifulSoup` para raspar detalhes do perfil (jogos, histórico de partidas). Em seguida, aplicar um modelo de IA (ex: `transformers` BERT) para classificar se o conteúdo textual do perfil é relevante às preferências do usuário (por exemplo, buscando termos de jogos citados pelo usuário). Mostrar se há “match” entre interesses do usuário e informações do perfil scraped.  

<div style="text-align: center;"> <h3>Estruturação do Notebook</h3> </div>

- Organizar o notebook em seções claras conforme as etapas acima: **Coleta de Dados**, **Validação de Identidade**, **Integração de Redes Sociais**, **Enriquecimento com Dados Sociais**, **Conclusão**.  
- Incluir explicações breves em cada seção usando células Markdown, resumindo o objetivo daquela etapa. Combinar descrições em texto com células de código demonstrativas.  
- Sugerir bibliotecas específicas no contexto de cada etapa: por exemplo, mencionar `ipywidgets` ou `streamlit` na coleta de dados, `pytesseract`/`OpenCV` na validação de documentos, `tweepy`/`PRAW`/`BeautifulSoup` na integração social, e `transformers`/`spaCy` na análise de linguagem.

<div style="text-align: center;"> <h3>Dicas de Apresentação do Protótipo</h3> </div>

- **Formatação Atraente:** Usar cabeçalhos (`#`, `##`), listas e imagens (logotipos de e-sports, ícones de redes sociais) para tornar o notebook visualmente agradável. Células Markdown bem elaboradas ajudam na legibilidade.  
- **Interatividade:** Incluir elementos interativos (sliders, botões de upload, caixas de seleção) via `ipywidgets` para simular um fluxo real de uso. Isso torna a demonstração dinâmica mesmo no ambiente de notebook.  
- **Visualização de Dados:** Aproveitar gráficos (matplotlib, seaborn ou plotly) para mostrar perfis de interesse ou resultados das análises de comentários. Um gráfico de barras ou nuvem de palavras torna o conteúdo mais didático.  
- **Narração do Código:** Inserir comentários explicativos e outputs de exemplo que guiem o avaliador pelo processo passo a passo. Ao final, apresentar um breve resumo dos resultados obtidos para evidenciar que todos os requisitos foram atendidos.  

<div style="text-align: center;"> <h3>Conclusão</h3> </div>

Este plano garante uma implementação completa dos requisitos do desafio, integrando coleta de informações pessoais e de interesse em e-sports, validação de identidade baseada em IA, simulação de integração social e enriquecimento de perfil com dados externos. A organização em seções claras, o uso de bibliotecas especializadas (ex: **Streamlit/ipywidgets** para interfaces, **OpenCV/face_recognition** para validação, **transformers/spaCy** para IA, **pandas** para dados) e as sugestões de apresentação asseguram uma entrega alinhada e de fácil acompanhamento, mesmo no formato de notebook.